https://cloud.pupil-labs.com/workspace/9ae5e987-5235-486f-8552-427014bbedcd/recordings

In [2]:
#pupil labs libs
from pupil_labs.realtime_api.simple import Device
import nest_asyncio

#multiprocessing
from pynput import keyboard
from multiprocessing import Process, Event, Value
from time import time

nest_asyncio.apply()

In [3]:
ip = "10.5.50.141"
device = Device(address=ip, port="8080")

In [4]:
class AcquisitionLogic:
    def __init__(self) -> None:
        #processes
        self.recording_process = Event()
        self.trigger_event_process = Event()

        self.event_i = Value('i', 0)
        #constants
        self.t_record =  60 #60 seconds of recording to test
        self.t_event = 2.0 #this is the event of gaze
        self.record_key = keyboard.Key.up
        self.event_key = keyboard.Key.right

    def on_press(self, key):

        if key is self.record_key:
            if self.recording_process.is_set():
                raise Exception('Wait! Recording is currently activated.')
            #else start recording process
            self.recording_process = Process(target=self.start_record_process)
            return
        
        if key is self.event_key:
            if not self.recording_process.is_set():
                raise Exception('Need to record to be able to trigger events')
            #else trigger event
            self.trigger_event_process = Process(target=self.start_trigger_event_process, args=(self.event_i.value,))

    def start_record_process(self):
        #assertions
        freestorage = device.memory_num_free_bytes
        battery = device.battery_level_percent
        assert (freestorage > 5e9), "Not enough free storage on device" #at least 5GB space
        assert (battery > 0.1), "Battery is too low" #at least 10% battery

        recording_id = device.start_recording()
        print(f'Started recording event. ({self.t_record} s)')
        time.sleep(self.t_record)
        device.recording_stop_and_save()
        print(f'Ended recording event.')

        

    def start_trigger_event_process(self, event_id = 0):
        #assertions
        assert self.recording_process.is_set(), "Need to record to be able to trigger events"
        #set event
        device.send_event(event_id, event_timestamp_unix_ns=time.time_ns())
        print(f'Triggered Event. ({self.t_event} s)')
        self.event_i.value += 1

In [5]:
acquisition_logic = AcquisitionLogic()
with keyboard.Listener(on_press=acquisition_logic.on_press) as listener:
    listener.join()
